In [1]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_ALL, '') 
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# ax1.spines['top'].set_visible(False) 

import os
# import csv
import math

import datetime
from datetime import date


# ax.legend(bbox_to_anchor=(1.04,0.5), loc="center left")

saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [2]:
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"
path_figs = os.getcwd() +"\\..\\Figures\\" 

# path_figs = path_figs + "Immunity_SSI\\"

In [3]:
latestsubdir = list(os.walk(path_dash))[0][1][-1]
latestdir = path_dash + latestsubdir
df = pd.read_csv(latestdir+'/Regionalt_DB/18_fnkt_alder_uge_testede_positive_nyindlagte.csv',encoding='latin1',delimiter = ';')

df['Nyindlagte pr. 100.000 borgere'] = pd.to_numeric(df['Nyindlagte pr. 100.000 borgere'].str.replace(',','.'))

# df_07['Dato'] = pd.to_datetime(df_07['Dato'])
df.tail()

,Uge,Aldersgruppe,Testede pr. 100.000 borgere,Positive pr. 100.000 borgere,Nyindlagte pr. 100.000 borgere,Antal testede,Antal positive
944,2022-W07,16-19,9247,3874,16.4,25351,10620
945,2022-W07,20-39,14799,4960,38.6,220251,73828
946,2022-W07,40-64,16852,4656,33.8,318352,87963
947,2022-W07,65-79,10700,3253,78.3,96711,29404
948,2022-W07,80+,12203,2346,193.5,35629,6850


In [4]:
weekShort = df.Uge.apply(lambda x: x[-2:])
# dfDate
df['Date'] = df.Uge.apply(lambda x: datetime.datetime.strptime(x + '-1', "%Y-W%W-%w"))

In [5]:
curdf = df[df.Aldersgruppe == '00-02']
curdf.tail()

,Uge,Aldersgruppe,Testede pr. 100.000 borgere,Positive pr. 100.000 borgere,Nyindlagte pr. 100.000 borgere,Antal testede,Antal positive,Date
904,2022-W03,00-02,14563,3541,84.8,27143,6600,2022-01-17
913,2022-W04,00-02,11454,3273,80.5,21349,6100,2022-01-24
922,2022-W05,00-02,9631,3331,97.6,17955,6210,2022-01-31
931,2022-W06,00-02,8608,3202,123.9,16048,5970,2022-02-07
940,2022-W07,00-02,5483,2126,114.3,10222,3963,2022-02-14


In [6]:
# curAdmCount

In [7]:
curAdm = curdf['Nyindlagte pr. 100.000 borgere']
curCase = curdf['Antal positive']
curInci = curdf['Positive pr. 100.000 borgere']
curPopCount = np.mean((100000*curCase/curInci)[-5:])
curPopCount 

curAdmCount = curPopCount*curAdm/100000

fig,allAxes = plt.subplots(2,1,sharex=True,figsize=(13,12))
# ax1,ax2,ax3 = allAxes.flatten()
ax1,ax2 = allAxes.flatten()

ax1.plot(curdf.Date,curAdm,'k.-')
ax1_2 = ax1.twinx()
ax1_2.plot(curdf.Date,curAdmCount,'k.-')

# ax1.set_ylabel('Nyindlæggelser per 100.000')
# ax1_2.set_ylabel('Antal nyindlæggelser')
ax1.set_ylabel('New admissions per 100.000')
ax1_2.set_ylabel('New admissions')

ax1.set_ylim(bottom=0)
ax1_2.set_ylim(bottom=0)

ax2.plot(curdf.Date,curdf['Positive pr. 100.000 borgere'],'k.-')
ax2_2 = ax2.twinx()
ax2_2.plot(curdf.Date,curdf['Antal positive'],'k.-')

ax2.set_ylabel('Cases per 100.000')
ax2_2.set_ylabel('Cases')
ax2.set_ylim(bottom=0)
ax2_2.set_ylim(bottom=0)

ax1.grid(axis='y')
ax2.grid(axis='y')

ax1.set_title('0 to 2 year olds')

if saveFigures:
    fig.savefig(path_figs+'CasesAndHospitalizations_Full_0-2')

ax1.set_xlim(left=np.datetime64('2021-09-01'))


if saveFigures:
    fig.savefig(path_figs+'CasesAndHospitalizations_0-2')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
# print(curAdmCount[:-1].values/curCase[1:].values)
# print(curAdmCount.values/curCase.values)
print(curCase.values[-5:])
print(curAdmCount.values[-5:])
print(curCase[2:].values[-5:])
print(curAdmCount[:-2].values[-5:])

[6600 6100 6210 5970 3963]
[158.07470537 150.05912479 181.93503826 230.96056598 213.06531632]
[6600 6100 6210 5970 3963]
[ 71.95381636 106.99868029 158.07470537 150.05912479 181.93503826]


In [9]:
caseHosp = curAdmCount/curCase
caseHospm1 = curAdmCount[1:].values/curCase[:-1].values
caseHospm2 = curAdmCount[2:].values/curCase[:-2].values

fig,ax1 = plt.subplots()
ax1.plot(curdf.Date,caseHosp,'k.:',markersize=2,linewidth=0.5,label='Admission divided by cases (same week)')
ax1.plot(rnTime(curdf.Date,4),rnMean(caseHosp,4),'k',label='4-week running mean')
ax1.plot(curdf.Date[:-1],caseHospm1,'r.:',markersize=2,linewidth=0.5,label='Admission divided by cases (week earlier)')
ax1.plot(rnTime(curdf.Date[:-1],4),rnMean(caseHospm1,4),'r',label='4-week running mean')
ax1.plot(curdf.Date[:-2],caseHospm2,'b.:',markersize=2,linewidth=0.5,label='Admission divided by cases (two weeks earlier)')
ax1.plot(rnTime(curdf.Date[:-2],4),rnMean(caseHospm2,4),'b',label='4-week running mean')

ax1.set_ylim(bottom=0)
ax1.set_ylim(top=0.4)

ax1.legend(loc='upper right',fontsize=13)
ax1.grid()

ax1.set_ylabel('Case hospitalization rate')
ax1.set_title('0 to 2 year olds')

if saveFigures:
    fig.savefig(path_figs+'CaseHospitalization_Full_0-2')

ax1.set_ylim(top=0.14)
ax1.set_xlim(left=np.datetime64('2021-09-01'))
if saveFigures:
    fig.savefig(path_figs+'CaseHospitalization_0-2')

<ipython-input-9-6fda3a493c24>:2: RuntimeWarning: divide by zero encountered in true_divide
  caseHospm1 = curAdmCount[1:].values/curCase[:-1].values
<ipython-input-9-6fda3a493c24>:2: RuntimeWarning: invalid value encountered in true_divide
  caseHospm1 = curAdmCount[1:].values/curCase[:-1].values
<ipython-input-9-6fda3a493c24>:3: RuntimeWarning: divide by zero encountered in true_divide
  caseHospm2 = curAdmCount[2:].values/curCase[:-2].values
<ipython-input-9-6fda3a493c24>:3: RuntimeWarning: invalid value encountered in true_divide
  caseHospm2 = curAdmCount[2:].values/curCase[:-2].values


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# # Collect age-specific cases for all municipalities
# allSubDirs = list(os.walk(path_dash))[0][1]

# df = pd.DataFrame()

# for curSubDir in allSubDirs:
#     curdir = path_dash + curSubDir
#     curfilepath = curdir+'/Regionalt_DB/21_noegletal_pr_region_pr_fnkt_alder.csv'

#     # Check if file was included at the time. The "Kommune/17" file wasn't included until 2021-09-22 
#     if os.path.isfile(curfilepath):
#         curdf = pd.read_csv(curfilepath,encoding='latin1',delimiter = ';')
        
#         df = pd.concat([df,curdf])

# df
# # # Set dtypes
# # df.Kommune = df.Kommune.fillna(0)  # All NaN kommuner is set to zero
# # df['Kommune'] = df['Kommune'].astype(int)
# # df['Dagsdato'] = pd.to_datetime(df['Dagsdato'])
# # df['Bekræftede tilfælde'] = pd.to_numeric(df['Bekræftede tilfælde'])
# # df['Aldersgruppe'] = df.Aldersgruppe.replace('00-02','0-2')
# # df['Aldersgruppe'] = df.Aldersgruppe.replace('03-05','3-5')
# # df['Aldersgruppe'] = df.Aldersgruppe.replace('06-11','6-11')

# # # df['Forskel'] = df['Bekræftede tilfælde'].diff().fillna(0).astype(int,errors='ignore')